In [2]:
from modules import device, model, context_input, response_input, embs_gen, cand_emb_gen, score, get_cand_embs, response_table

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
cand_embs = get_cand_embs()

In [4]:
cand_embs

tensor([[[ 0.4922, -0.6527,  0.3834,  ..., -0.2355, -0.0816,  0.7831],
         [ 0.4265, -0.7164,  0.4097,  ..., -0.4494,  0.0708,  1.1537],
         [ 0.3295, -0.6451,  0.3668,  ..., -0.4229,  0.1024,  1.1591],
         ...,
         [ 0.7648, -0.7776,  0.2120,  ..., -0.2492,  0.2255,  0.7545],
         [ 0.0553, -0.9839,  0.3826,  ..., -0.7569, -0.0208,  0.4652],
         [ 0.4029, -0.7795,  0.4000,  ..., -0.3931,  0.0195,  0.8489]]])

In [5]:
response_table

,context,response
0,"['participant 1: 아내가 드디어 출산하게 되어서 정말 신이 나.', '...",['좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.']
1,['participant 1: 당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어...,['약을 잘 챙겨 먹어 건강해지시길 바랄게요.']
2,['participant 1: 고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려...,['열심히 해서 좋은 결과가 있었으면 좋겠어요.']
3,['participant 1: 재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을...,['계속 원하시는 대로 회사생활이 이어지길 바랄게요.']
4,['participant 1: 빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.'...,['빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.']
...,...,...
40874,['participant 1: 같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌...,['친구에게 얘기해서 혼자 부담을 떠안지 않으면 좋겠어요.']
40875,['participant 1: 지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 ...,['건강검진에서 좋은결과를 받으신 상황이군요.']
40876,['participant 1: 엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 ...,['꿈을 어떻게 이룰 것인지 엄마에게 이야기해보려 하시는군요.']
40877,['participant 1: 이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있...,['안마기로 피로가 많이 풀리시길 바라요.']
